# Investment and Renewable Energy Support

This week we are looking at investment in electricity markets. Specifically, we will look at how we can understand and model investment incentives, and how policy can influence these. We will first look at a simple way of thinking about investment, using screening curves. We will then see an example of a more complex modelling method which can include much more detail.

## Screening curves

Electricity demand fluctuates from hour to hour. Because of this, an optimal electricity generation mix, i.e. the particular combination of power plants that can serve the market, will likely consist of a number of different types of power plants. Some will be used most of the time - these are base load power plants, which generally have high investment costs but low marginal cost. Then there are mid-load plants, which are used often but not all the time (with medium investment costs and medium marginal costs) and peak load power plants which are only used when demand is very high. Peak load power plants generally have a low investment cost, but a high marginal cost. An optimal (i.e. lowest total cost) generation mix generally has all these types, because it is not worth building expensive base load power plants to cover all demand (as some of them will not be used all the time), but it’s also a waste of money to only use peak load plants as they might be cheap to build, but if you’re running them all the time this is false economy, as their running costs are high.

Screening curves can visualise the trade-off between investment and operational costs and show what the optimal mix of technologies might look like. Before we look at one, we need a representation of energy demand. A time series of electricity demand might look like this (I have plotted 8760 hours here for demonstration only - of course for real-world investment purposes you’d want data to cover several years, although in larger models it is common to cluster that data to save computation time):

In [1]:
import pandas as pd
import datetime
import numpy as np
import hvplot.pandas
import holoviews as hv
hvplot.extension('plotly')

import warnings
warnings.filterwarnings("ignore")

# Sample data (replace this with your actual data)
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='H')  # Use hourly frequency
hours_in_day = 24

# Simulate higher demand in winter than summer and daily peaks
base_demand = [
    120 + 10 * np.sin((i % (365 * hours_in_day)) * 2 * np.pi / (365 * hours_in_day)) +
    30 * ((7 <= i % hours_in_day <= 11) or (17 <= i % hours_in_day <= 20))
    for i in range(len(date_range))
]

# Introduce random noise
np.random.seed(42)  # Set seed for reproducibility
noise = np.random.normal(loc=0, scale=5, size=len(date_range))  # Adjust the scale for the amount of noise
demand_with_noise = base_demand + noise

data = {'Datetime': date_range, 'Electrical Demand': demand_with_noise}

df = pd.DataFrame(data)

# Create a line chart using hvplot
hv_plot = df.hvplot(
    kind='line',
    x='Datetime',
    y='Electrical Demand',
    label='Electrical Demand',
    title='Electrical Demand Over a Year with Seasonal and Daily Peaks',
    xlabel='Datetime',
    ylabel='Demand (Units)', width=800, height=500,
    legend=False
)

# Show the plot
hv_plot

:Curve   [Datetime]   (Electrical Demand)

If we now sort this time series, arranging the 8760 hours in order from largest to smallest demand levels, we obtain a load duration curve:

In [9]:
import plotly.express as px
import pandas as pd
import datetime
import numpy as np
import hvplot.pandas
import holoviews as hv
hvplot.extension('plotly')

# Sample data (replace this with your actual data)
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='H')  # Use hourly frequency
hours_in_day = 24

# Simulate higher demand in winter than summer and daily peaks
base_demand = [
    120 + 10 * np.sin((i % (365 * hours_in_day)) * 2 * np.pi / (365 * hours_in_day)) +
    30 * ((7 <= i % hours_in_day <= 11) or (17 <= i % hours_in_day <= 20))
    for i in range(len(date_range))
]

# Introduce random noise
np.random.seed(42)  # Set seed for reproducibility
noise = np.random.normal(loc=0, scale=5, size=len(date_range))  # Adjust the scale for the amount of noise
demand_with_noise = base_demand + noise

data = {'Datetime': date_range, 'Electrical Demand': demand_with_noise}

df = pd.DataFrame(data)

# Sort the data by demand in descending order
df = df.sort_values(by='Electrical Demand', ascending=False)

# Calculate the cumulative percentage of time
df['Cumulative Percentage'] = (df['Electrical Demand'].rank(ascending=False, method='max') / len(df)) * 100

# # Create a load duration curve using Plotly Express
# fig = px.line(df, x='Cumulative Percentage', y='Electrical Demand', title='Load Duration Curve',
#               labels={'Electrical Demand': 'Demand (Units)', 'Cumulative Percentage': 'Cumulative Percentage'})
# Create a line chart using hvplot
hv_plot = df.hvplot(
    kind='line',
    x='Cumulative Percentage',
    y='Electrical Demand',
    label='Electrical Demand',
    title='Load Duration Curve',
    xlabel='Cumulative Percentage',
    ylabel='Electrical Demand', width=800, height=500,
    legend=False
)

# Show the plot
hv_plot

:Curve   [Cumulative Percentage]   (Electrical Demand)

We can now combine this load duration curve with a screening curve, which is simply a plot of the total (investment + fuel) cost of different generation technologies as a function of the number of hours the technologies are used. Consider the example below:

In [16]:
import plotly.graph_objects as go

# Data for different thermal power plant technologies
full_load_hours = [i for i in range(0, 8001, 1000)]  # 0 to 8000 in increments of 1000

# Linear equation for total annual cost based on run hours
def calculate_costs(run_hours, intercept, slope):
    return [intercept + slope * hours for hours in run_hours]

# Parameters for each technology
natural_gas_intercept = 100
natural_gas_slope = 0.06

coal_intercept = 150
coal_slope = 0.04

nuclear_intercept = 300
nuclear_slope = 0.01

# Calculate total annual costs for each technology
natural_gas_costs = calculate_costs(full_load_hours, natural_gas_intercept, natural_gas_slope)
coal_costs = calculate_costs(full_load_hours, coal_intercept, coal_slope)
nuclear_costs = calculate_costs(full_load_hours, nuclear_intercept, nuclear_slope)

# Create traces for each technology
trace_gas = go.Scatter(x=full_load_hours, y=natural_gas_costs, mode='lines+markers', name='Natural Gas')
trace_coal = go.Scatter(x=full_load_hours, y=coal_costs, mode='lines+markers', name='Coal')
trace_nuclear = go.Scatter(x=full_load_hours, y=nuclear_costs, mode='lines+markers', name='Nuclear')

# Create layout
layout = go.Layout(
    title='Screening Curves for Thermal Power Plant Technologies',
    xaxis=dict(title='Full Load Hours'),
    yaxis=dict(title='Total Annual Costs'),
    legend=dict(x=0, y=1),
)

# Create figure
fig = go.Figure(data=[trace_gas, trace_coal, trace_nuclear], layout=layout)
fig

In [21]:
import holoviews as hv
from holoviews import opts

# Create HoloViews objects for each technology
curve_gas = hv.Curve((full_load_hours, natural_gas_costs), 'Full Load Hours', 'Total Annual Costs', label='Natural Gas')
curve_coal = hv.Curve((full_load_hours, coal_costs), 'Full Load Hours', 'Total Annual Costs', label='Coal')
curve_nuclear = hv.Curve((full_load_hours, nuclear_costs), 'Full Load Hours', 'Total Annual Costs', label='Nuclear')

# Combine the curves into a single overlay
overlay = curve_gas * curve_coal * curve_nuclear

# Set the options for the plot
overlay.opts(
    opts.Curve(line_width=2),
    opts.Overlay(title='Screening Curves for Thermal Power Plant Technologies', width=800, height=500)
)

# Display the plot
hv.extension('plotly')
overlay

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

:Overlay
   .Curve.Natural_Gas :Curve   [Full Load Hours]   (Total Annual Costs)
   .Curve.Coal        :Curve   [Full Load Hours]   (Total Annual Costs)
   .Curve.Nuclear     :Curve   [Full Load Hours]   (Total Annual Costs)

The example above had no renewables, but these can easily be included as long as it is never optimal to curtail renewables. Renewable production can simply be subtracted from demand to generate a net demand curve, which will lead to a different optimal conventional generation mix. Existing generation capacity can also be included - this would have a zero investment cost (as it already exist).

## Investment stage in market models

Screening curves are a good first approximation to analyse optimal investment in electricity markets - and therefore also investment levels we might see in a perfectly competitive market. However, there are many things that cannot be included in screening curves. Because the demand data is sorted to obtain the load duration curve, all information about the temporal sequence of time period is lost. Technologies such as storage, which shift demand from one time period to the next, are therefore difficult to include, as are ramping constraints, start-ups, etc. Grid constraints, which can limit how power plants can be used, can also not be included. 

For a more detailed analysis of investment in electricity markets (and therefore, for instance, also the effect of policies that target investment), it is possible to extend the market models that we have seen before to include an investment stage. In the widget below, you can explore this.

Experiment with this. Where does investment take place, depending on your chosen parameters?

In [4]:
from IPython.display import IFrame
IFrame('https://eee-apps-836a08e1d2dd.herokuapp.com/electricity-investment-1', width='100%', height=710)

Of course, this model is very simple. A more realistic model could include
- multiple time periods, as in the first example on this notebook
- some of the technical constraints on generators that we have explored in previous weeks.
- a demand function, instead of perfectly inelastic demand. This would make the model quadratic, as the objective would now be a maximisation of social welfare.
- a representation of uncertainty about fuel costs, demand, etc., e.g., by having a set of scenarios for each of these parameters, with the objective minimising expected costs. This would make it a stochastic model.
- multiple investment stages (e.g., with investment decisions happening every few years), which gives the model the option to not invest right now but wait and see what will happen to some of the uncertain parameters. This would make it a multi-stage model.
- a measure of risk aversion, such that the objective would not be to minimise expected costs but some risk metric. This would make it a stochastic riskaverse model or, if we care about expected costs but want the model to come up with a feasible solution in all cases, a robust model.
- more levels, e.g. an explicit representation of the system operator engaging in redispatch; line outages, etc,
- game theory elements, e.g. different investors in a market that is imperfectly competitive. This is harder, as we then need to solve a number of optimisation models simultaneously, making the problem an equilibrium problem.

The flexible nature of these type of models mean that almost anything can be included (but if you make models too large and complex, they will become difficult or impossible to solve, and interpreting the results will be difficult in any case!). There is specialist optimisation software (e.g., GAMS, AIMMS, Fico Xpress, etc.) that can solve these problems efficiently; in addition, more and more packages for open-source alternatives such Python and Julia are being released, which can help build and solve a model quickly.

## Renewable Energy Support Schemes

Please see the material here: http://www.open-electricity-economics.org/book/text/08.html#renewable-energy-support-schemes. We will run through this in class - but this is a fantastic resource to explore.

## Renewable Energy Auction Design

Please see the material here: http://www.open-electricity-economics.org/book/text/09.html#renewable-energy-auction-design. We will run through this in class - but this is a fantastic resource to explore.